In [ ]:
import os
import cv2

IMG_DIR = 'images'
OUTPUT_DIR = 'augmentation_images'


def load_images_from_folder(folder_path, ext_list=['.jpg', '.jpeg', '.png', '.bmp', 'tif']):
    images = []
    file_names = []

    for filename in os.listdir(folder_path):
        if any(filename.lower().endswith(ext) for ext in ext_list):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            images.append(img)
            file_names.append(filename[:12])
    return images, file_names

folder_path = "images"
images, file_names = load_images_from_folder(folder_path)

print(f"{len(images)} adet resim yüklendi.")


In [ ]:
import numpy as np
import pandas as pd
import cv2

df = pd.read_csv("data.csv")

label_dict = dict(zip(df["Patient"], df["histological_type"]))


rows = []

for img_bgr, file_name in zip(images, file_names):
    row= {}
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    label = label_dict[file_name]  

    row["label"] = label
    row["filename"] = file_name
    rows.append(row)

feature_df = pd.DataFrame(rows)


In [ ]:
import os
import pandas as pd
from PIL import Image, ImageOps, ImageEnhance
from tqdm import tqdm


os.makedirs(OUTPUT_DIR, exist_ok=True)

all_images = os.listdir(IMG_DIR)

augmented_rows = []

for i, row in tqdm(feature_df.iterrows(), total=len(feature_df)):
    short_name = row['filename']
    label = row['label']

    matched_files = [f for f in all_images if f.startswith(short_name)]

    if not matched_files:
        print(f"Eşleşen dosya bulunamadı: {short_name}")
        continue

    for full_filename in matched_files:
        try:
            img_path = os.path.join(IMG_DIR, full_filename)
            img = Image.open(img_path).convert("RGB")

            original_save_path = os.path.join(OUTPUT_DIR, full_filename)
            img.save(original_save_path)
            augmented_rows.append({'filename': full_filename, 'label': label})

            flipped = ImageOps.mirror(img)
            flip_name = f"{full_filename[:-4]}_flip.jpg"
            flipped.save(os.path.join(OUTPUT_DIR, flip_name))
            augmented_rows.append({'filename': flip_name, 'label': label})

            rotated = img.rotate(15)
            rot_name = f"{full_filename[:-4]}_rot.jpg"
            rotated.save(os.path.join(OUTPUT_DIR, rot_name))
            augmented_rows.append({'filename': rot_name, 'label': label})

            contrast_img = ImageEnhance.Contrast(img).enhance(1.5)
            contrast_name = f"{full_filename[:-4]}_contrast.jpg"
            contrast_img.save(os.path.join(OUTPUT_DIR, contrast_name))
            augmented_rows.append({'filename': contrast_name, 'label': label})

        except Exception as e:
            print(f"Hata: {full_filename} işlenemedi: {e}")

aug_df = pd.DataFrame(augmented_rows)
print(f"Toplam {len(aug_df)} resim augment edildi.")
